In [1]:
%load_ext rich

# LLM Provider
A provider is a class that implements `ProviderProtocol`, serving as an interface to interact with a large language model (LLM). It processes user queries and returns model-generated responses.

```python
class LLMProtocol(Protocol):
    async def achat(self, history: list[Message]) -> Message: ...
```

Expected Behavior

-  Implements `achat(self, history: list[Message]) -> Message`
-  Accepts a conversation history (list of `Message` objects)
-  Returns a single assistant-generated response (`Message`)

A provider facilitates structured communication with an LLM, ensuring consistency in message handling and response generation.

In [2]:
import promptimus as pm

In [8]:
# creating a provider
#
# do not forgent to export OPENAI_API_KEY with your token or pass it as api-key argument
llm = pm.llms.OpenAILike(
    model_name="gemma3:4b", base_url="http://lilan:11434/v1", api_key="DUMMY"
)

In [4]:
# using a provider with single message
await llm.achat(
    [
        pm.Message(role=pm.MessageRole.USER, content="Hi, who are you?"),
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="Hi there! I'm Gemma, a large language model created by the Gemma team at Google DeepMind. I'm an open-weights model, which means I’m widely available for public use! \n\nI can take text and images as inputs and generate text-based responses. \n\nYou can learn more about me on the Gemma project page: [https://ai.google.com/gemma](https://ai.google.com/gemma) \n\nWhat would you like to do?",
    images=[],
    tool_calls=None,
    tool_call_id=None
)

In [5]:
# using provider with multiple messages
await llm.achat(
    [
        pm.Message(role=pm.MessageRole.SYSTEM, content="Your name is Mark."),
        pm.Message(role=pm.MessageRole.USER, content="Hi, what is your name?"),
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='Hi there! My name is Mark. It’s nice to meet you. 😊 \n\nHow can I help you today?',
    images=[],
    tool_calls=None,
    tool_call_id=None
)

![](https://picsum.photos/id/42/200/)

In [6]:
import io

import requests

## Image inputs

In [9]:
image_file = requests.get("https://picsum.photos/id/42/200/")
image_buffer = io.BytesIO(image_file.content)

await llm.achat(
    [
        pm.Message(
            role=pm.MessageRole.USER,
            content="brief description of an image",
            images=[pm.ImageContent.from_buffer(image_buffer, "image/jpeg")],
        ),
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="Here's a brief description of the image:\n\nThe image shows a rustic, wooden table in what appears to be a cafe or coffee shop. There are two mugs of coffee and a phone on the table. In the background, there are other patrons seated at a similar table, along with a window showing an exterior view. The lighting is warm, creating a cozy atmosphere.",
    images=[],
    tool_calls=None,
    tool_call_id=None
)

## OpenAI like 

In [17]:
import asyncio

from tqdm.auto import tqdm

In [10]:
openai = pm.llms.OpenAILike(
    model_name="gpt-5-nano",
    max_concurrency=50,
)

In [11]:
await openai.achat(
    [
        pm.Message(
            role=pm.MessageRole.USER,
            content="brief description of an image",
            images=[pm.ImageContent(url="https://picsum.photos/id/42/200/")],
        ),
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='A cozy cafe interior with a long, rustic wooden table in the foreground. A smartphone sits on the table beside a green cup on a saucer, while soft natural light pours in from large windows and a few people are seen at tables in the background.',
    images=[],
    tool_calls=None,
    tool_call_id=None
)

### Rate limiting

In [9]:
for task in tqdm.as_completed(
    [
        openai.achat(
            [
                Message(role=MessageRole.USER, content="Hi, who are you?"),
            ]
        )
        for _ in range(1_000)
    ]
):
    await task

  0%|          | 0/1000 [00:00<?, ?it/s]

2025-06-18 08:58:45.859 | WARNING  | promptimus.common.rate_limiting:execute_request:54 - OpenAILike rate limit hit (attempt 1/5), waiting 3.000s (base 3.000s, exponential backoff)
2025-06-18 08:58:53.300 | WARNING  | promptimus.common.rate_limiting:execute_request:54 - OpenAILike rate limit hit (attempt 2/5), waiting 6.000s (base 3.000s, exponential backoff)
2025-06-18 08:59:00.150 | WARNING  | promptimus.common.rate_limiting:execute_request:54 - OpenAILike rate limit hit (attempt 3/5), waiting 12.000s (base 3.000s, exponential backoff)
2025-06-18 08:59:12.747 | INFO     | promptimus.common.rate_limiting:execute_request:47 - OpenAILike rate limit resolved.
2025-06-18 08:59:12.815 | INFO     | promptimus.common.rate_limiting:execute_request:47 - OpenAILike rate limit resolved.
2025-06-18 08:59:14.470 | WARNING  | promptimus.common.rate_limiting:execute_request:54 - OpenAILike rate limit hit (attempt 4/5), waiting 24.000s (base 3.000s, exponential backoff)
2025-06-18 08:59:39.045 | INFO

# Embedder
A embedder is a class that implements `EmbedderProtocol`, serving as an interface to generate text embeddings.

```python
Embedding = list[float]


class EmbedderProtocol(Protocol):
    async def aembed_batch(self, texts: list[str], **kwargs: Any) -> list[Embedding]: ...
    async def aembed(self, text: str, **kwargs: Any) -> Embedding: ...
```

In [10]:
import promptimus as pm

In [11]:
embedder = pm.embedders.OpenAILikeEmbedder(
    model_name="mxbai-embed-large", base_url="http://lilan:11434/v1", api_key="DUMMY"
)

In [12]:
embedding = await embedder.aembed("Hi")
len(embedding), embedding[:3]

(1024, [0.049403064, 0.012764112, -0.006060004])

In [13]:
embeddings = await embedder.aembed_batch(["Hi", "How are you?"])

len(embeddings)

2